# 뉴스기사 전처리 후 저장하기

In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [168]:
import re
from pykospacing import Spacing
from konlpy.tag import Hannanum,Kkma,Komoran,Okt

In [169]:
import FinanceDataReader as fdr

In [170]:
# 모든 날짜 채워주기 (2016-01-01 ~ 2022-03-31)
def fill_date(df):
    
    # 날짜로 묶기
    df = df.groupby('날짜')['뉴스제목'].sum().to_frame()
    
    # 모든 날짜 채우기
    idx = pd.date_range('2016-01-01','2022-03-31')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value= '')
    
    return df

In [171]:
# 한글,영어만 남기기
def clean_text(docs):
    docs = re.sub('[^가-힣A-Za-z ]', '', docs)
    docs = re.sub('\s+', ' ', docs)
    docs = '' if docs== ' ' else docs
    return docs

In [172]:
# 띄어쓰기
def spacing_text(docs):
    spacing = Spacing()
    docs = spacing(docs)
    return docs

In [173]:
# 명사 추출
def han_noun(docs):
    han = Hannanum()
    docs = han.nouns(docs)
    return docs

In [174]:
# 불용어 제거+ 한글자 이상만 남기기
stw_list = pd.read_csv('./data/stopwords-ko.txt')
def remove_stwords(docs):
    docs = [w for w in docs if not w in stw_list]
    docs = '' if docs== ' ' else docs
    docs = [w for w in docs if len(w)>1]
    return docs

In [25]:
df = pd.read_csv('./data/개별기업뉴스_전처리/AJ네트웍스.csv',index_col=0)
print(len(df))
df.head()

907


,날짜,뉴스제목,뉴스요약
1313,2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스가 SK브로드밴드를 통해 진행중인 '스마트렌탈'의 위약금 규정 및 취급상...
1312,2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,..."
1311,2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","국내 IT기기 렌탈의 선두주자인 AJ네트웍스에서 Mobile Device(스마트폰,..."
1332,2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스(사장 반채운)가 파렛트와 고소장비 렌탈 부문의 신규 서비스 브랜드를 런...
1335,2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,신용등급이 BBB+인 AJ네트웍스도 회사채 발행을 위해 지난 20일 투자자들을 대상...


In [27]:
df = fill_date(df)
df.head()

,뉴스제목
2016-01-01,
2016-01-02,
2016-01-03,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'"
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작"


In [4]:
df_n = df.groupby('날짜')['뉴스제목'].sum().to_frame()

In [11]:
df_n['뉴스'] = df_n['뉴스제목'].apply(clean_text)
df_n.head()

,뉴스제목,뉴스
날짜,,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'",AJ네트웍스 SKB 등에업고 고리사채급 위약금 폭탄
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작",AJ네트웍스 스마트폰 태블릿PC 렌탈사업 시작
2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작",AJ네트웍스 스마트폰 태블릿PC 등 Mobile Device 렌탈 사업 시작
2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭",AJ네트웍스 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭
2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,대상AJ네트웍스 등 비우량채도 잇달아 발행 성공


In [13]:
df_n['뉴스'] = df_n['뉴스'].apply(spacing_text)
df_n['뉴스'].head()

날짜
2016-01-04                 AJ네트웍스 SKB 등에 업고 고리 사채급 위약금 폭탄
2016-01-05                     AJ네트웍스 스마트폰 태블릿PC 렌탈 사업 시작
2016-01-06    AJ네트웍스 스마트폰 태블릿PC 등 Mobile Dev ice 렌탈 사업 시작
2016-01-13             AJ네트웍스 파렛트 및 고소장비 렌탈 신규 서비스 브랜드 런칭
2016-01-22                    대상 AJ네트웍스 등 비우량채도 잇달아 발행 성공
Name: 뉴스, dtype: object

In [19]:
df_n['뉴스'] = df_n['뉴스'].apply(han_noun)
df_n.head()

,뉴스제목,뉴스
날짜,,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'","[AJ네트웍스, 등, 사채급, 위약금, 폭탄]"
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","[AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]"
2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","[AJ네트웍스, 스마트폰, 태블릿PC, 등, 렌탈, 사업, 시작]"
2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭","[AJ네트웍스, 파렛트, 고소장비, 렌탈, 신규, 서비스, 브랜드, 런칭]"
2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,"[대상, AJ네트웍스, 등, 우량채, 발행, 성공]"


In [21]:
df_n['뉴스'] = df_n['뉴스'].apply(remove_stwords)
df_n.head()

,뉴스제목,뉴스
날짜,,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'","[AJ네트웍스, 사채급, 위약금, 폭탄]"
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작","[AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]"
2016-01-06,"AJ네트웍스, 스마트폰, 태블릿PC 등 Mobile Device 렌탈 사업 시작","[AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]"
2016-01-13,"AJ네트웍스, 파렛트및 고소장비 렌탈 신규 서비스 브랜드 런칭","[AJ네트웍스, 파렛트, 고소장비, 렌탈, 신규, 서비스, 브랜드, 런칭]"
2016-01-22,대상·AJ네트웍스 등 비우량채도 잇달아 발행 성공,"[대상, AJ네트웍스, 우량채, 발행, 성공]"


In [22]:
bgn_date = '2016-01-01'
end_date = '2016-02-01'

token_lst = df_n['뉴스'].loc[bgn_date:end_date]
token_lst

날짜
2016-01-04                       [AJ네트웍스, 사채급, 위약금, 폭탄]
2016-01-05            [AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]
2016-01-06            [AJ네트웍스, 스마트폰, 태블릿PC, 렌탈, 사업, 시작]
2016-01-13    [AJ네트웍스, 파렛트, 고소장비, 렌탈, 신규, 서비스, 브랜드, 런칭]
2016-01-22                    [대상, AJ네트웍스, 우량채, 발행, 성공]
Name: 뉴스, dtype: object

In [24]:
kw_dict = dict()
for tokens in token_lst:

    for word in tokens:
        if not word in kw_dict.keys():
            kw_dict[word] = 1
        else:
            kw_dict[word] += 1

kw_dict = dict(sorted(kw_dict.items(), key = lambda x: x[1]))

kw_dict

{'사채급': 1,
 '위약금': 1,
 '폭탄': 1,
 '파렛트': 1,
 '고소장비': 1,
 '신규': 1,
 '서비스': 1,
 '브랜드': 1,
 '런칭': 1,
 '대상': 1,
 '우량채': 1,
 '발행': 1,
 '성공': 1,
 '스마트폰': 2,
 '태블릿PC': 2,
 '사업': 2,
 '시작': 2,
 '렌탈': 3,
 'AJ네트웍스': 5}

# 합친코드

In [37]:
df = pd.read_csv('./data/개별기업뉴스_전처리/AJ네트웍스.csv',index_col=0)
df = fill_date(df)

In [38]:
df.head()

,뉴스제목
2016-01-01,
2016-01-02,
2016-01-03,
2016-01-04,"AJ네트웍스, SKB 등에업고 '고리사채급 위약금 폭탄'"
2016-01-05,"AJ네트웍스, 스마트폰, 태블릿PC 렌탈사업 시작"


In [31]:
path = './data/개별기업뉴스_전처리/'
file_list = os.listdir(path)
for file in file_list[:3]:
    corp_name = file[:-4]
    print(corp_name)

AJ네트웍스
AK홀딩스
BGF


In [53]:
db_list = pd.read_csv('./data/개별기업명_511',index_col=0)
db_list.head()

,stock_code,회사이름
0,40,KR모터스
1,50,경방
2,80,하이트진로
3,100,유한양행
4,120,CJ대한통운


## 완성코드1

In [154]:
# 완성코드

db_list = pd.read_csv('./data/개별기업명_511.csv',index_col=0)
path = './data/개별기업뉴스_전처리/'
file_list = os.listdir(path)

for file in file_list[:1]:
    corp_name = file[:-4]
    print(corp_name)
    if corp_name in db_list['회사이름'].values:
        file_path = os.path.join(path,file)
        df = pd.read_csv(file_path,index_col=0)
        df = fill_date(df)
        df['뉴스'] = df['뉴스제목'].apply(clean_text)
#         df['뉴스'] = df['뉴스제목'].apply(spacing_text)
        df['뉴스'] = df['뉴스'].apply(han_noun)
        df['뉴스'] = df['뉴스'].apply(remove_stwords)
        df.to_csv('./data/개별기업뉴스_자연어처리/{}.csv'.format(corp_name))

AJ네트웍스


## 완성코드 2: spacing 포함

In [166]:
# 완성코드

db_list = pd.read_csv('./data/개별기업명_511.csv',index_col=0)
path = './data/개별기업뉴스_전처리/'
file_list = os.listdir(path)

for file in file_list[:1]:
    corp_name = file[:-4]
    print(corp_name)
    if corp_name in db_list['회사이름'].values:
        file_path = os.path.join(path,file)
        df = pd.read_csv(file_path,index_col=0)
        df = fill_date(df)
        df['뉴스'] = df['뉴스제목'].apply(clean_text)
        df['뉴스'] = df['뉴스'].apply(spacing_text)
        df['뉴스'] = df['뉴스'].apply(han_noun)
        df['뉴스'] = df['뉴스'].apply(remove_stwords)
        df.to_csv('./data/개별기업뉴스_자연어처리_2/{}.csv'.format(corp_name))

AJ네트웍스


KeyboardInterrupt: 

## 완성코드: 누락파일

In [9]:
x_list = pd.read_csv('./data/누락기업40.csv',index_col=0)
path = './data/개별기업뉴스_전처리/'
file_list = os.listdir(path)

for file in file_list[:]:
    corp_name = file[:-4]
    if corp_name in x_list['회사이름'].values:
        print(corp_name)
        
        file_path = os.path.join(path,file)
        df = pd.read_csv(file_path,index_col=0)
        df = fill_date(df)
        df['뉴스'] = df['뉴스제목'].apply(clean_text)
        df['뉴스'] = df['뉴스'].apply(han_noun)
        df['뉴스'] = df['뉴스'].apply(remove_stwords)
        df.to_csv('./data/개별기업뉴스_자연어처리/{}.csv'.format(corp_name))

동원F&B
무림P&P
세이브존I&C


## 37개 기업

In [180]:
path = './data/추가37_전처리/'
file_list = os.listdir(path)
len(file_list)

37

In [181]:
for file in file_list:
    corp_name = file[:-4]
    file_path = os.path.join(path,file)
    df = pd.read_csv(file_path,index_col=0)
    df.columns=['날짜','뉴스제목','뉴스요약']
    df = fill_date(df)
    df['뉴스'] = df['뉴스제목'].apply(clean_text)
    df['뉴스'] = df['뉴스'].apply(han_noun)
    df['뉴스'] = df['뉴스'].apply(remove_stwords)
    df.to_csv('./data/추가37_자연어처리/{}.csv'.format(corp_name))

## 용량적은거

In [ ]:
path = './data/temp/'
file_list = os.listdir(path)
for file in file_list:
    corp_name = file[:-4]
    file_path = os.path.join(path,file)
    df = pd.read_csv(file_path,index_col=0)
    df.columns=['날짜','뉴스제목','뉴스요약']
    df = fill_date(df)
    df['뉴스'] = df['뉴스제목'].apply(clean_text)
    df['뉴스'] = df['뉴스'].apply(han_noun)
    df['뉴스'] = df['뉴스'].apply(remove_stwords)
    df.to_csv('./data//{}.csv'.format(corp_name))

In [45]:
db_list = pd.read_csv('./data/DB_list_stock_code.csv',index_col=0)
db_list['stock_code'] = db_list['stock_code'].apply(lambda x: str(x).zfill(6))
db_list

,stock_code
0,000040
1,000050
2,000080
3,000100
4,000120
...,...
506,214330
507,214420
508,226320
509,227840


In [50]:
len(corp_list)

511

# 누락기업 자연어처리

In [11]:
df_x = pd.read_csv('./data/누락기업40.csv',index_col=0)
df_x.head()

,회사이름
13,삼화페인트
50,DI동일
59,삼화콘덴서
68,한국수출포장
80,아모레G


In [12]:
# 완성코드
db_list = pd.read_csv('./data/개별기업명_511.csv',index_col=0)
path = './data/개별기업뉴스_전처리/'
file_list = os.listdir(path)
file_list

['AJ네트웍스.csv',
 'AK홀딩스.csv',
 'BGF.csv',
 'BGF리테일.csv',
 'BNK금융지주.csv',
 'BYC.csv',
 'CJ CGV.csv',
 'CJ.csv',
 'CJ대한통운.csv',
 'CJ씨푸드.csv',
 'CJ제일제당.csv',
 'CS홀딩스.csv',
 'DB.csv',
 'DB금융투자.csv',
 'DB손해보험.csv',
 'DB하이텍.csv',
 'DGB금융지주.csv',
 'DL.csv',
 'DL건설.csv',
 'DL이앤씨.csv',
 'DRB동일.csv',
 'DSR.csv',
 'DSR제강.csv',
 'E1.csv',
 'ESR켄달스퀘어리츠.csv',
 'F&F 홀딩스.csv',
 'F&F.csv',
 'GKL.csv',
 'GS.csv',
 'GS건설.csv',
 'GS글로벌.csv',
 'GS리테일.csv',
 'HDC.csv',
 'HDC랩스.csv',
 'HDC현대EP.csv',
 'HDC현대산업개발.csv',
 'HD현대.csv',
 'HJ중공업.csv',
 'HLB글로벌.csv',
 'HMM.csv',
 'HSD엔진.csv',
 'JB금융지주.csv',
 'JW생명과학.csv',
 'JW중외제약.csv',
 'JW홀딩스.csv',
 'KB금융.csv',
 'KC그린홀딩스.csv',
 'KC코트렐.csv',
 'KEC.csv',
 'KG스틸.csv',
 'KG케미칼.csv',
 'KH 필룩스.csv',
 'KISCO홀딩스.csv',
 'KPX케미칼.csv',
 'KPX홀딩스.csv',
 'KR모터스.csv',
 'KSS해운.csv',
 'KT.csv',
 'KTis.csv',
 'LF.csv',
 'LG.csv',
 'LG디스플레이.csv',
 'LG생활건강.csv',
 'LG에너지솔루션.csv',
 'LG유플러스.csv',
 'LG이노텍.csv',
 'LG전자.csv',
 'LG헬로비전.csv',
 'LG화학.csv',
 'LIG넥스원.csv',
 'LS.csv',
 'LS네트웍스.csv

In [21]:
df511 = pd.Series(file_list).to_frame()
df511.head()

,0
0,AJ네트웍스.csv
1,AK홀딩스.csv
2,BGF.csv
3,BGF리테일.csv
4,BNK금융지주.csv


In [55]:
# 완성코드
df_820 = pd.read_csv('./data/개별기업공시_전처리',index_col=0)

path = './data/개별기업공시_전처리/'
file_list = os.listdir(path)
file_list

# 누락40
x_list = pd.read_csv('./data/누락기업40.csv',index_col=0)

df_x1 = pd.DataFrame()
for file in file_list:
    corp = file[:-4]
    
    for corp in x_list['회사이름'].tolist() :
        
        df_xx = df_820[df_820['회사이름'].str.contains(corp)]
        df_x1 = pd.concat([df_x1,df_xx])
df_x1

PermissionError: [Errno 13] Permission denied: './data/개별기업공시_전처리'

In [51]:
x_list['회사이름'].tolist()[0]

'삼화페인트'

In [69]:
pd.Series(file_list)[pd.Series(file_list).str.contains('쌍용')]

440    쌍용씨앤이.csv
441    쌍용자동차.csv
dtype: object

In [30]:
# 완성코드: 누락파일
x_list = pd.read_csv('./data/누락기업40.csv',index_col=0)
x_list

,회사이름
13,삼화페인트
50,DI동일
59,삼화콘덴서
68,한국수출포장
80,아모레G
101,쌍용C&E
105,IHQ
107,쌍용차
109,삼영화학
118,한국석유


In [ ]:
for file in file_list[:]:
    corp_name = file[:-4]
    if corp_name in x_list['회사이름'].values:
        print(corp_name)
        
        file_path = os.path.join(path,file)
        df = pd.read_csv(file_path,index_col=0)
        df = fill_date(df)
        df['뉴스'] = df['뉴스제목'].apply(clean_text)
        df['뉴스'] = df['뉴스'].apply(han_noun)
        df['뉴스'] = df['뉴스'].apply(remove_stwords)
        df.to_csv('./data/개별기업뉴스_자연어처리/{}.csv'.format(corp_name))

In [74]:
df_c = pd.read_csv('./data/누락기업40수정.csv',index_col=0).dropna()
len(df_c)

29

## 820개 기업이름

### 기존파일: df_kospi

In [104]:
df_kospi = pd.read_csv('./data/kospi_list.csv',index_col=0)
ser11 = pd.Series(df_kospi['corp_name'])

In [111]:
df_kospi[df_kospi['Name']=='삼화페인트']

,corp_code,corp_name,stock_code,modify_date,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
748,129387,삼화페인트공업,390,20220512,390,KOSPI,삼화페인트,기타 화학제품 제조업,"건축용페인트,공업용페인트,분체도료 등 제조",1993-09-10,12월,"오진수,류기붕(각자대표)",http://www.samhwa.com,경기도


In [112]:
df_kospi[df_kospi['Name']=='삼화페인트공업']

,corp_code,corp_name,stock_code,modify_date,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region


### 현재 fdr: df_820

In [98]:
df_krx = fdr.StockListing('KRX')
df_kospi1 = df_krx[df_krx['Market']=='KOSPI']
df_kospi1

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
6,152100,KOSPI,ARIRANG 200,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7,295820,KOSPI,ARIRANG 200동일가중,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8,253150,KOSPI,ARIRANG 200선물레버리지,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
8240,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8241,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8242,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
8244,419890,KOSPI,히어로즈 단기채권ESG액티브,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [117]:
df_820 = df_kospi1.dropna(subset=['Sector'])
df_820.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
54,027410,KOSPI,BGF,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
55,282330,KOSPI,BGF리테일,종합 소매업,체인화 편의점,2017-12-08,12월,이건준,http://www.bgfretail.com,서울특별시
57,138930,KOSPI,BNK금융지주,기타 금융업,금융지주회사,2011-03-30,12월,김지완,http://www.bnkfg.com,부산광역시


In [113]:
df_kospi1[df_kospi1['Name']=='삼화페인트']

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
4453,000390,KOSPI,삼화페인트,기타 화학제품 제조업,"건축용페인트,공업용페인트,분체도료 등 제조",1993-09-10,12월,"오진수,류기붕(각자대표)",http://www.samhwa.com,경기도


In [118]:
ser_r = pd.Series(df_820['Name'])

### 이름 다른 기업 모음 : df_ya

In [124]:
df_ya = ser11[~ser11.isin(ser_r)].to_frame()
df_ya.head()

,corp_name
9,태양금속공업
41,진양폴리우레탄
48,효성 ITX
62,삼영화학공업
77,하이트론씨스템즈


In [125]:
len(ser11[~ser11.isin(ser_r)]) 

54

In [138]:
code_list = []
for name in df_ya['corp_name'].tolist():
    if name in df_kospi['corp_name'].tolist():
        code = df_kospi[df_kospi['corp_name']==name]['Symbol'].iloc[0]
        code_list.append(code)
code_list 

[4100,
 10640,
 94280,
 3720,
 19490,
 1200,
 3410,
 11230,
 970,
 1820,
 1530,
 344820,
 11780,
 2200,
 810,
 4250,
 5300,
 33780,
 33270,
 25540,
 326030,
 3620,
 5870,
 17390,
 7700,
 4410,
 6060,
 84870,
 900140,
 5680,
 3560,
 123700,
 5750,
 10100,
 13580,
 25530,
 11700,
 3650,
 10420,
 181710,
 30200,
 17800,
 53210,
 334890,
 2380,
 5380,
 69260,
 4090,
 10120,
 9070,
 390,
 15760,
 35510,
 2790]

In [130]:
df_ya['corp_name'].tolist()[0]

'태양금속공업'

In [139]:
len(code_list)

54

In [140]:
df_ya['code'] = code_list

In [143]:
df_ya.to_csv('./data/수정해야할기업54.csv')